In [33]:
%load_ext autoreload
%aimport os, pandas, numpy, pickle
%autoreload 2

In [34]:
import sys
import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble.forest import ExtraTreesRegressor

from fqi.et_tuning import run_tuning
from fqi.reward_function import *
from fqi.sars_creator import to_SARS
from fqi.utils import *

from trlib.policies.valuebased import EpsilonGreedy, Softmax
from trlib.policies.qfunction import ZeroQ
from trlib.algorithms.reinforcement.fqi_driver import FQIDriver, DoubleFQIDriver
from trlib.environments.trackEnv import TrackEnv
from trlib.utilities.ActionDispatcher import *

from fqi.fqi_evaluate import run_evaluation

In [59]:
simulations = pd.read_csv('trajectory/dataset_offroad.csv')
ref_tr = pd.read_csv('./trajectory/ref_traj.csv')

In [60]:
NLap = 12

In [61]:
file_name = 'model_file/penalty/penalty.pkl'
with open('./' + file_name, 'rb') as pen:
     penalty = pickle.load(pen)
        
file_name = 'model_file/penalty/penalty_offroad.pkl'
with open('./' + file_name, 'rb') as pen:
     penalty_offroad = pickle.load(pen)

In [62]:
rf = Temporal_projection(ref_tr, penalty=penalty, clip_range=(-np.inf, np.inf))
rf_off = Temporal_projection(ref_tr, penalty=penalty, clip_range=(-np.inf, np.inf), offroad = True)

In [63]:
data = pd.read_csv('./trajectory/dataset_offroad.csv',
                              dtype={'isReference': bool, 'is_partial':bool})
NLap = 12
data = data[data['NLap'] == NLap]
episodes = {}
for e in np.unique(data['NLap']):
    mask = data['NLap'] == e
    lap_df = data[mask]
    n_samples = np.count_nonzero(mask)
    # create timestamp column
    timestamp = pd.DataFrame({'t': np.zeros([n_samples - 1])})
    # Create NLap column
    nlap = pd.DataFrame({'NLap': np.ones([n_samples - 1]) * e}, dtype=int)
    # Add state and action
    state_action = lap_df[state_cols+action_cols].iloc[:-1].reset_index(drop=True)
    # Add reward
    #reward = pd.DataFrame({'r': reward_function(lap_df)})
    r = rf_off(lap_df)
    #print(r)
    print('type:', type(r))
    print('shape', r.shape)

type: <class 'numpy.ndarray'>
shape (849,)


In [55]:
rf_off